<h1 style="color: teal; font-size: 40px;">❤️ Heart Disease Classification Project</h1>

In [31]:
# classification_project.py

import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [32]:
# Load dataset
df = pd.read_csv("heart.csv")
X = df.drop("target", axis=1)
y = df["target"]

In [33]:
# Preprocessing
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [34]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [35]:
# Models
models = {
    "Logistic Regression": LogisticRegression(),
    "KNN": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC(probability=True)
}

In [36]:
# Train and evaluate
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    results[name] = {
        "accuracy": accuracy_score(y_test, preds),
        "precision": precision_score(y_test, preds),
        "recall": recall_score(y_test, preds),
        "f1_score": f1_score(y_test, preds)
    }

In [37]:
# Print results
results_df = pd.DataFrame(results).T.sort_values(by="f1_score", ascending=False)
print(results_df)

                     accuracy  precision    recall  f1_score
Decision Tree        0.985366   1.000000  0.970874  0.985222
Random Forest        0.985366   1.000000  0.970874  0.985222
SVM                  0.887805   0.850877  0.941748  0.894009
KNN                  0.834146   0.800000  0.893204  0.844037
Logistic Regression  0.795122   0.756303  0.873786  0.810811


In [38]:
# Save best model
best_model_name = results_df.index[0]
best_model = models[best_model_name]
with open("heart_model.pkl", "wb") as f:
    pickle.dump(best_model, f)

print(f"\nSaved best model: {best_model_name}")



Saved best model: Decision Tree


<h1 style="color: teal; font-size: 40px;">GUI</h1>

In [40]:
import pickle
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load saved model
with open("heart_model.pkl", "rb") as file:
    model = pickle.load(file)

# Feature input widgets (based on UCI Heart dataset)
age = widgets.IntSlider(min=25, max=80, value=50, description='Age')
sex = widgets.Dropdown(options=[('Male', 1), ('Female', 0)], description='Sex')
cp = widgets.Dropdown(options=[('Typical Angina', 0), ('Atypical Angina', 1), 
                                ('Non-anginal Pain', 2), ('Asymptomatic', 3)], description='Chest Pain Type')
trestbps = widgets.IntSlider(min=80, max=200, value=120, description='Resting BP')
chol = widgets.IntSlider(min=100, max=600, value=240, description='Cholesterol')
fbs = widgets.Dropdown(options=[('> 120 mg/dl', 1), ('<= 120 mg/dl', 0)], description='Fasting Blood Sugar')
restecg = widgets.Dropdown(options=[('Normal', 0), ('ST-T abnormality', 1), 
                                     ('Left ventricular hypertrophy', 2)], description='Rest ECG')
thalach = widgets.IntSlider(min=70, max=210, value=150, description='Max Heart Rate')
exang = widgets.Dropdown(options=[('Yes', 1), ('No', 0)], description='Exercise Induced Angina')
oldpeak = widgets.FloatSlider(min=0.0, max=6.0, value=1.0, step=0.1, description='Oldpeak')
slope = widgets.Dropdown(options=[('Upsloping', 0), ('Flat', 1), ('Downsloping', 2)], description='Slope')
ca = widgets.IntSlider(min=0, max=3, value=0, description='CA (vessels)')
thal = widgets.Dropdown(options=[('Normal', 1), ('Fixed Defect', 2), ('Reversible Defect', 3)], description='Thal')

predict_btn = widgets.Button(description="💉 Predict", button_style='success')
output_box = widgets.Output()

# Prediction logic
def predict_heart_disease(b):
    with output_box:
        clear_output()
        input_data = np.array([[
            age.value, sex.value, cp.value, trestbps.value, chol.value,
            fbs.value, restecg.value, thalach.value, exang.value,
            oldpeak.value, slope.value, ca.value, thal.value
        ]])
        prediction = model.predict(input_data)[0]
        result = "⚠️ High Risk of Heart Disease" if prediction == 1 else "✅ Low Risk of Heart Disease"
        color = "red" if prediction == 1 else "green"
        display(widgets.HTML(f"<h3 style='color:{color}'>{result}</h3>"))

predict_btn.on_click(predict_heart_disease)

# GUI Layout
gui = widgets.VBox([
    widgets.HTML("<h2 style='color:darkred;'>🫀 Heart Disease Predictor</h2>"),
    age, sex, cp, trestbps, chol, fbs, restecg,
    thalach, exang, oldpeak, slope, ca, thal,
    predict_btn, output_box
])

display(gui)
